In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pysentiment2 as ps
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import gensim
from sklearn.feature_extraction.text import CountVectorizer
import yfinance as yf
from tqdm import tqdm
from tqdm.auto import tqdm
from textblob import TextBlob

def sentiment_vader(var):
    #pip install vaderSentiment
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
    sentiment = SentimentIntensityAnalyzer() 
    out_score = sentiment.polarity_scores(var)
    return out_score
def sentiment_ps(var):
    lm = ps.LM()
    tokens = lm.tokenize(var)
    score = lm.get_score(tokens)
    return score
def sentiment_tb(var):
    text = TextBlob(var)
    return text.sentiment.polarity
def topic_distribution(string_input):
    string_input = [string_input]
    # Fit and transform
    X = vect.transform(string_input)
 
    # Convert sparse matrix to gensim corpus.
    corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
 
    output = list(ldamodel[corpus])[0]
 
    return output
def topic_prediction(my_document):
    string_input = [my_document]
    X = vect.transform(string_input)
    # Convert sparse matrix to gensim corpus.
    corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
    output = list(ldamodel[corpus])[0]
    topics = sorted(output,key=lambda x:x[1],reverse=True)
    return topics[0][0]
def normalize(arr):
     return (arr-min(arr))/(max(arr)-min(arr))
def normalize_2(arr):
     return np.interp(arr, (arr.min(), arr.max()), (-1, +1))

In [46]:
df = pd.read_pickle("sentiment.pkl")
df['Year']= pd.to_datetime(df['Year'])
df = df.set_index('Year')
df['Year'] = df.index
df

,Statement,polarity,positive,negative,subjectivity,Year
Year,,,,,,
1993-02-03,"The Federal Reserve, the central bank of the U...",-0.258278,112,190,0.075368,1993-02-03
1993-03-23,A meeting of the Federal Open Market Committee...,-0.299492,69,128,0.081304,1993-03-23
1993-05-18,A meeting of the Federal Open Market Committee...,-0.114504,116,146,0.080591,1993-05-18
1993-07-07,A meeting of the Federal Open Market Committee...,-0.203488,137,207,0.070680,1993-07-07
1993-08-17,A meeting of the Federal Open Market Committee...,-0.159574,79,109,0.084229,1993-08-17
...,...,...,...,...,...,...
2020-09-16,"The Federal Reserve, the central bank of the U...",-0.239560,173,282,0.067298,2020-09-16
2020-11-05,"The Federal Reserve, the central bank of the U...",-0.238771,161,262,0.080771,2020-11-05
2020-12-16,"The Federal Reserve, the central bank of the U...",-0.100977,138,169,0.070737,2020-12-16


In [49]:
df = pd.read_csv("mth_data_michelle.csv")
df

,date,stock_idx_dv,ebitda_dv,bus_conf,cpi,vix,inv
0,2021-03-01,-0.013697,0.4347,0.002259,0.006163,-0.059400,0.022862
1,2021-02-01,-0.014997,0.4347,0.002715,0.003534,-0.076466,0.022862
2,2021-01-01,-0.009599,0.4347,0.001514,0.002559,0.101803,0.022862
3,2020-12-01,-0.170073,-0.0212,0.001919,0.002420,-0.117163,-0.006085
4,2020-11-01,-0.163321,-0.0212,0.001498,0.001782,-0.177757,-0.006085
...,...,...,...,...,...,...,...
622,1969-05-01,-0.045051,NaN,-0.000665,0.002747,NaN,NaN
623,1969-04-01,-0.052058,NaN,-0.000847,0.005510,NaN,NaN
624,1969-03-01,-0.018934,NaN,0.000048,0.008310,NaN,NaN
625,1969-02-01,-0.017834,NaN,0.000602,0.002793,NaN,NaN


In [47]:
df

,Statement,polarity,positive,negative,subjectivity,Year
Year,,,,,,
1993-02-03,"The Federal Reserve, the central bank of the U...",-0.258278,112,190,0.075368,1993-02-03
1993-03-23,A meeting of the Federal Open Market Committee...,-0.299492,69,128,0.081304,1993-03-23
1993-05-18,A meeting of the Federal Open Market Committee...,-0.114504,116,146,0.080591,1993-05-18
1993-07-07,A meeting of the Federal Open Market Committee...,-0.203488,137,207,0.070680,1993-07-07
1993-08-17,A meeting of the Federal Open Market Committee...,-0.159574,79,109,0.084229,1993-08-17
...,...,...,...,...,...,...
2020-09-16,"The Federal Reserve, the central bank of the U...",-0.239560,173,282,0.067298,2020-09-16
2020-11-05,"The Federal Reserve, the central bank of the U...",-0.238771,161,262,0.080771,2020-11-05
2020-12-16,"The Federal Reserve, the central bank of the U...",-0.100977,138,169,0.070737,2020-12-16


In [50]:
df.date = pd.to_datetime(df.date).dt.to_period('m')

In [51]:
df

,date,stock_idx_dv,ebitda_dv,bus_conf,cpi,vix,inv
0,2021-03,-0.013697,0.4347,0.002259,0.006163,-0.059400,0.022862
1,2021-02,-0.014997,0.4347,0.002715,0.003534,-0.076466,0.022862
2,2021-01,-0.009599,0.4347,0.001514,0.002559,0.101803,0.022862
3,2020-12,-0.170073,-0.0212,0.001919,0.002420,-0.117163,-0.006085
4,2020-11,-0.163321,-0.0212,0.001498,0.001782,-0.177757,-0.006085
...,...,...,...,...,...,...,...
622,1969-05,-0.045051,NaN,-0.000665,0.002747,NaN,NaN
623,1969-04,-0.052058,NaN,-0.000847,0.005510,NaN,NaN
624,1969-03,-0.018934,NaN,0.000048,0.008310,NaN,NaN
625,1969-02,-0.017834,NaN,0.000602,0.002793,NaN,NaN


In [39]:
fed = pd.read_pickle("sentiment.pkl")
fed['Year']= pd.to_datetime(fed['Year'])
fed

,Year,Statement,polarity,positive,negative,subjectivity
0,1993-02-03,"The Federal Reserve, the central bank of the U...",-0.258278,112,190,0.075368
1,1993-03-23,A meeting of the Federal Open Market Committee...,-0.299492,69,128,0.081304
2,1993-05-18,A meeting of the Federal Open Market Committee...,-0.114504,116,146,0.080591
3,1993-07-07,A meeting of the Federal Open Market Committee...,-0.203488,137,207,0.070680
4,1993-08-17,A meeting of the Federal Open Market Committee...,-0.159574,79,109,0.084229
...,...,...,...,...,...,...
220,2020-09-16,"The Federal Reserve, the central bank of the U...",-0.239560,173,282,0.067298
221,2020-11-05,"The Federal Reserve, the central bank of the U...",-0.238771,161,262,0.080771
222,2020-12-16,"The Federal Reserve, the central bank of the U...",-0.100977,138,169,0.070737
223,2021-01-27,"The Federal Reserve, the central bank of the U...",-0.137778,194,256,0.069930


In [40]:
fed.Year = pd.to_datetime(fed.Year).dt.to_period('m')

In [41]:
fed

,Year,Statement,polarity,positive,negative,subjectivity
0,1993-02,"The Federal Reserve, the central bank of the U...",-0.258278,112,190,0.075368
1,1993-03,A meeting of the Federal Open Market Committee...,-0.299492,69,128,0.081304
2,1993-05,A meeting of the Federal Open Market Committee...,-0.114504,116,146,0.080591
3,1993-07,A meeting of the Federal Open Market Committee...,-0.203488,137,207,0.070680
4,1993-08,A meeting of the Federal Open Market Committee...,-0.159574,79,109,0.084229
...,...,...,...,...,...,...
220,2020-09,"The Federal Reserve, the central bank of the U...",-0.239560,173,282,0.067298
221,2020-11,"The Federal Reserve, the central bank of the U...",-0.238771,161,262,0.080771
222,2020-12,"The Federal Reserve, the central bank of the U...",-0.100977,138,169,0.070737
223,2021-01,"The Federal Reserve, the central bank of the U...",-0.137778,194,256,0.069930


In [42]:
pd.__version__

'1.2.3'

In [ ]:
df.join(other.set_index('key'), on='key')

In [52]:
df = df.drop(['ebitda_dv'], axis=1)

In [53]:
df

,date,stock_idx_dv,bus_conf,cpi,vix,inv
0,2021-03,-0.013697,0.002259,0.006163,-0.059400,0.022862
1,2021-02,-0.014997,0.002715,0.003534,-0.076466,0.022862
2,2021-01,-0.009599,0.001514,0.002559,0.101803,0.022862
3,2020-12,-0.170073,0.001919,0.002420,-0.117163,-0.006085
4,2020-11,-0.163321,0.001498,0.001782,-0.177757,-0.006085
...,...,...,...,...,...,...
622,1969-05,-0.045051,-0.000665,0.002747,NaN,NaN
623,1969-04,-0.052058,-0.000847,0.005510,NaN,NaN
624,1969-03,-0.018934,0.000048,0.008310,NaN,NaN
625,1969-02,-0.017834,0.000602,0.002793,NaN,NaN


In [54]:
df = df.dropna()

In [57]:
df

,date,stock_idx_dv,bus_conf,cpi,vix,inv
0,2021-03,-0.013697,0.002259,0.006163,-0.059400,0.022862
1,2021-02,-0.014997,0.002715,0.003534,-0.076466,0.022862
2,2021-01,-0.009599,0.001514,0.002559,0.101803,0.022862
3,2020-12,-0.170073,0.001919,0.002420,-0.117163,-0.006085
4,2020-11,-0.163321,0.001498,0.001782,-0.177757,-0.006085
...,...,...,...,...,...,...
232,2001-11,-0.467380,0.001551,-0.000563,-0.228694,-0.164170
233,2001-10,-0.452609,-0.003521,-0.002815,-0.071636,-0.164170
234,2001-09,-0.728210,-0.001602,0.003930,0.376543,0.049264
235,2001-08,-0.697551,0.002242,0.000000,-0.020788,0.049264


In [58]:
fed.shape

(225, 6)

In [59]:
joined = df.merge(fed, left_on='date', right_on='Year')

In [61]:
joined = joined.dropna()

In [63]:
joined = joined.drop(['Year', 'Statement', 'positive', 'negative', 'subjectivity'], axis=1)

In [64]:
joined

,date,stock_idx_dv,bus_conf,cpi,vix,inv,polarity
0,2021-03,-0.013697,0.002259,0.006163,-0.059400,0.022862,-0.488584
1,2021-01,-0.009599,0.001514,0.002559,0.101803,0.022862,-0.137778
2,2020-12,-0.170073,0.001919,0.002420,-0.117163,-0.006085,-0.100977
3,2020-11,-0.163321,0.001498,0.001782,-0.177757,-0.006085,-0.238771
4,2020-09,-0.050308,0.003041,0.002452,0.172098,0.024193,-0.239560
...,...,...,...,...,...,...,...
152,2002-01,-0.047997,0.006536,0.001688,-0.065931,0.067622,-0.320695
153,2001-12,-0.486871,0.004744,-0.000564,-0.122721,-0.164170,-0.358149
154,2001-11,-0.467380,0.001551,-0.000563,-0.228694,-0.164170,-0.358149
155,2001-10,-0.452609,-0.003521,-0.002815,-0.071636,-0.164170,-0.434090


In [73]:
y = joined.stock_idx_dv
Y = pd.DataFrame(y)

In [76]:
X = joined.drop(['stock_idx_dv','date'], axis=1)

In [77]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size= 0.2)

In [78]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(x_train, y_train)

<ipython-input-78-86687fbd2f52>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regr.fit(x_train, y_train)


RandomForestRegressor(max_depth=2, random_state=0)

In [80]:
print(regr.score(x_test,y_test))

-7.094187057911768
